In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


Note: you may need to restart the kernel to use updated packages.


In [1]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from google.oauth2 import service_account
import io

SCOPES = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/documents"]

SERVICE_ACCOUNT_FILE = "credentials.json"

def get_google_services():
    """Xác thực Google Docs & Google Drive API"""
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    
    docs_service = build("docs", "v1", credentials=creds)
    drive_service = build("drive", "v3", credentials=creds)
    
    return docs_service, drive_service


In [2]:
def create_google_doc(docs_service, text):
    """Tạo Google Docs mới và chèn văn bản"""
    document = docs_service.documents().create(body={"title": "Generated Document"}).execute()
    document_id = document["documentId"]

    # Chèn văn bản vào tài liệu
    requests = [{"insertText": {"location": {"index": 1}, "text": text}}]
    docs_service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()
    
    return document_id


In [3]:
def export_doc_to_pdf(drive_service, document_id):
    """Xuất Google Docs thành PDF và lấy link tải"""
    request = drive_service.files().export_media(fileId=document_id, mimeType="application/pdf")
    
    pdf_filename = f"{document_id}.pdf"
    with open(pdf_filename, "wb") as pdf_file:
        pdf_file.write(request.execute())

    return pdf_filename


In [4]:
def upload_pdf_to_drive(drive_service, pdf_filename):
    """Upload file PDF lên Google Drive và lấy link chia sẻ"""
    file_metadata = {"name": pdf_filename, "mimeType": "application/pdf"}
    media = MediaFileUpload(pdf_filename, mimetype="application/pdf")

    uploaded_file = drive_service.files().create(body=file_metadata, media_body=media, fields="id").execute()
    file_id = uploaded_file.get("id")

    # Cấp quyền xem file cho mọi người
    permission = {"role": "reader", "type": "anyone"}
    drive_service.permissions().create(fileId=file_id, body=permission).execute()

    # Lấy link truy cập file
    file_link = f"https://drive.google.com/file/d/{file_id}/view"
    return file_link


In [5]:
if __name__ == "__main__":
    docs_service, drive_service = get_google_services()
    
    # Nội dung cần xuất
    text_content = "This is a sample document generated using Google Docs API and converted to PDF."

    # Bước 1: Tạo Google Docs
    document_id = create_google_doc(docs_service, text_content)

    # Bước 2: Xuất file PDF
    pdf_file = export_doc_to_pdf(drive_service, document_id)

    # Bước 3: Upload lên Google Drive & Lấy link
    pdf_link = upload_pdf_to_drive(drive_service, pdf_file)

    print(f"✅ PDF Link: {pdf_link}")


✅ PDF Link: https://drive.google.com/file/d/1esw5wSZ9-eG6_0fDHWRvy5erGUUQm_W2/view
